In [3]:
from google.colab import files
files.upload()

Saving combine.xlsx to combine.xlsx


In [4]:
import pandas as pd
cf1=pd.read_csv("zxtrain.csv",index_col=False)
cf2=pd.read_csv("zxtest.csv",index_col=False)
cf2["review_comment_message"].fillna("nao_reveja",inplace=True)
cf3=pd.read_csv("zytrain.csv",index_col=False)
cf4=pd.read_csv("/content/zytest.csv",index_col=False)

In [5]:
ytest=list(cf4["class"].values)

In [6]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [7]:
def final_1(uploaded_file):
    count=0
    xls = pd.ExcelFile(uploaded_file)
    geo= pd.read_excel(xls,'geo',index_col="Unnamed: 0")
    item=pd.read_excel(xls,'item',index_col="Unnamed: 0")
    customer= pd.read_excel(xls,'customer',index_col="Unnamed: 0")
    review=pd.read_excel(xls,'review',index_col="Unnamed: 0")
    orders= pd.read_excel(xls,'orders',index_col="Unnamed: 0")
    products=pd.read_excel(xls,'products',index_col="Unnamed: 0")
    sellers= pd.read_excel(xls,'sellers',index_col="Unnamed: 0")
    payment=pd.read_excel(xls,'payment',index_col="Unnamed: 0")
    model=pickle.load(open('zstack_model.pkl', 'rb'))

    df=pd.merge(orders,payment,on="order_id",how="inner")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,customer,on="customer_id",how="outer")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,item,on="order_id",how="inner")
    df.drop_duplicates(inplace=True)
    df.drop_duplicates(subset=["order_id"],inplace=True)
    df=pd.merge(df,products,on="product_id",how="inner")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,review,on="order_id",how="outer")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,sellers,on="seller_id",how="outer")
    df.drop_duplicates(inplace=True)
    df.replace(to_replace="bebes",
              value="babies")
    df.replace(to_replace="ferramentas_jardim",
              value="garden_tools")
    df.replace(to_replace="informatica_acessorios",
              value="informatica_accessories")
    df["review_comment_message"].fillna("nao_reveja",inplace=True)
    df.drop(columns=["review_answer_timestamp","review_creation_date","review_id","review_comment_title","order_delivered_carrier_date"],axis=1,inplace=True)
    df1=df.copy()
    df1["review_score"]=df1["review_score"].apply(lambda x: 0 if x < 3 else 1)
    df1["order_purchase_timestamp"]=pd.to_datetime(df1["order_purchase_timestamp"])
    df1["order_approved_timestamp"]=pd.to_datetime(df1["order_approved_at"])
    df1["order_purchase_date"]=df1.order_purchase_timestamp.dt.date
    df1["approval_date"]=df1["order_approved_timestamp"].dt.date
    df1["order_estimated_delivery_date"]=pd.to_datetime(df1.order_estimated_delivery_date).dt.date
    df1["order_delivered_customer_date"]=pd.to_datetime(df1.order_delivered_customer_date).dt.date
    df1.drop_duplicates(subset=["order_id","customer_id","order_status","order_status","order_approved_at","order_delivered_customer_date","product_weight_g","product_category_name_english","review_comment_message"],inplace=True)
    print("data retained after doing doing data cleaning {:.1f}".format((df1.shape[0]/df.shape[0])*100))
    import nltk
    nltk.download("stopwords")
    from nltk.corpus import stopwords
    from tqdm import tqdm
    from nltk.stem import RSLPStemmer
    import re
    nltk.download("rslp")
    stopwords_portugues=stopwords.words("portuguese")
    stopwords_portugues.remove("nem")
    stopwords_portugues.remove("não")
    stemmer=RSLPStemmer()
    preprocessed_reviews = []
    for sentance in tqdm(df1['review_comment_message'].values):
        sentance = re.sub(r"http\S+", "", sentance)
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        sentance=re.sub("[nN][ãaAÃ][oO][Ññ]|[nN]","Negação",sentance)
        sentance=sentance.replace("\\r",'')
        sentance=sentance.replace("\\n",'')
        sentance = re.sub('[^A-Za-z]+',' ',sentance)
        sentance = ' '.join(a for a in sentance.split() if a.lower not in stopwords_portugues)
        sentance = ' '.join(stemmer.stem(e.lower())for e in sentance.split())
        
        preprocessed_reviews.append(sentance.strip())


    f10=df1.groupby("order_id").count()["product_id"]
    f10.df=pd.DataFrame()
    f10.df["order_id"]=f10.index
    f10.df["F10_number_of_product_ordered"]=f10.values
    f9=df1.groupby("product_id").count()["seller_id"]
    f9.df=pd.DataFrame()
    f9.df["product_id"]=f9.index
    f9.df["number_of_sellers"]=f9.values
    df1["f16_purchase_month"]=(df1["order_purchase_timestamp"]).apply(lambda x: x.month)
    df1["f14_delivery_days"]=(df1["order_delivered_customer_date"]-df1["order_purchase_date"]).dt.days
    df1=pd.merge(df1,f9.df,on="product_id")
    df1=pd.merge(df1,f10.df,on="order_id")

    df1["f1_delivery_estimated_days"]=(df1["approval_date"]-df1["order_estimated_delivery_date"]).dt.days
    df1["f2_delivery_actual_days"]=(df1["approval_date"]-df1["order_delivered_customer_date"]).dt.days
    df1["f3_on_time_or_late"]=(df1["f1_delivery_estimated_days"] > df1["f2_delivery_actual_days"]).astype("int")
    df1["f4_discount_or_not"]=(df1["payment_value"]-(df1["price"]+df1["freight_value"])).apply(lambda x: 1 if x < 0 else 0)#converted to boolean 
    df1["f5_purchase_approved_delay"]=(df1["approval_date"]-df1["order_purchase_date"]).dt.days
    df1["f6_Average_product_value_per_order"]=df1.price/(df1.F10_number_of_product_ordered)
    df1["f7_order_frieght_value"]=df1.freight_value/df1.price
    df1["f8_purchase_day_week"]=df1.order_purchase_timestamp.dt.dayofweek
    df1["f11_order_purch_day"]=df1.order_purchase_timestamp.dt.day
    df1["f12_words_per_review"]=df1.review_comment_message.apply(lambda x: len(x.split(" ")))
    df1["f13_order_purchase_hour"]=df1["order_purchase_timestamp"].apply(lambda x: x.hour)
    df1["f15_week_delivery"]=df1["order_purchase_timestamp"].apply(lambda x: x.dayofweek)


    df1["review_comment_message"]=preprocessed_reviews


    
    xtest=df1.copy()
    xtest.drop(['order_id'],axis=1,inplace=True)
    xtest=xtest[["product_description_lenght","product_name_lenght","order_status","payment_type","product_category_name_english","payment_installments","product_weight_g","price","payment_sequential","product_photos_qty","f13_order_purchase_hour","f6_Average_product_value_per_order","payment_value","f11_order_purch_day","f12_words_per_review",'f2_delivery_actual_days','f3_on_time_or_late', 'f4_discount_or_not','F10_number_of_product_ordered','review_score','review_comment_message']]
    xtedon=xtest[['f3_on_time_or_late', 'f4_discount_or_not']]
    
    if 0<xtest['f6_Average_product_value_per_order'].values<7000:
        norm=pickle.load(open('znorm1.pkl', 'rb'))
        xte_apv=norm.transform(xtest['f6_Average_product_value_per_order'].values.reshape(-1,1))
        count+=1
    else:
        print("enter correct value of average product value per order between 0-7000")
        xte_apv=0
    
    
    from sklearn.feature_extraction.text import CountVectorizer
    
    if 3 < xtest["product_description_lenght"].values < 4000:
        norm=pickle.load(open('znorm3.pkl', 'rb'))
        xte_pdl=norm.transform(xtest["product_description_lenght"].values.reshape(-1,1))
        count+=1
    else:
         print("enter correct product description length between 3-4000")
         xte_pdl=0
        
    if 4 < xtest["product_name_lenght"].values < 80:
        norm=pickle.load(open('znorm4.pkl', 'rb'))
        xte_pnl=norm.transform(xtest["product_name_lenght"].values.reshape(-1,1))
        count+=1
    
    else:
      print("enter correct product name length between 4-80")
      xte_pnl=0   
   
    if 0 < xtest["payment_value"].values < 14000:
        norm=pickle.load(open('znorm5.pkl', 'rb'))
        xte_pv=norm.transform(xtest["payment_value"].values.reshape(-1,1))
        count+=1
    else:
        print("enter valid payment value between 0-14000")
        xte_pv=0
    
    if 0 < xtest["price"].values < 7000:
          norm=pickle.load(open('znorm7.pkl', 'rb'))
          xte_pri=norm.transform(xtest['price'].values.reshape(-1,1))
          count+=1
    else:
       print("enter the correct price between 0-7000")
       xte_pri=0
    

    if 0 < xtest['product_weight_g'].values < 41000:

      norm=pickle.load(open('znorm8.pkl', 'rb'))
      xte_pw=norm.transform(xtest['product_weight_g'].values.reshape(-1,1))
      count+=1
    
    else:
       print("enter correct product weight between 0-41000")
       xte_pw=0  
    
    if 0<=xtest['payment_installments'].values<=30:
      norm=pickle.load(open('znorm9.pkl', 'rb'))
      xte_pi=norm.transform(xtest['payment_installments'].values.reshape(-1,1))
      count+=1
    else:
      print("correct payment insatllments 0-30")
      xte_pi=0

    #here we are normalising the numerical features as so that all come to same scale
    norm=pickle.load(open('znorm10.pkl', 'rb'))
    xte_dad=norm.transform(xtest["f2_delivery_actual_days"].values.reshape(-1,1))
     

    norm=pickle.load(open('norm11.pkl', 'rb'))
    xte_opd=norm.transform(xtest["f11_order_purch_day"].values.reshape(-1,1))
    
    if 0< xtest["f12_words_per_review"].values <60:
      norm=pickle.load(open('znorm12.pkl', 'rb'))
      xte_wpr=norm.transform(xtest["f12_words_per_review"].values.reshape(-1,1))
      count+=1
    else:
      print("enter limited words in review comment message between  0-60 ")
      xte_wpr=0
    
    
    if 0< xtest['F10_number_of_product_ordered'].values < 10:

      norm=pickle.load(open('znorm13.pkl', 'rb'))
      xte_po=norm.transform(xtest['F10_number_of_product_ordered'].values.reshape(-1,1))
      count+=1
    else:
      print("enter correct number of products ordered between 0-10 ")
      xte_po=0


    vect=pickle.load(open('zvect.pkl', 'rb'))
    xte_os=vect.transform(xtest["order_status"].values)



    vect=pickle.load(open('zvect1.pkl', 'rb'))
    xte_pty=vect.transform(xtest["payment_type"].values)


    vect=pickle.load(open('zvect2.pkl', 'rb'))
    xte_pcm=vect.transform(xtest["product_category_name_english"].values)

    norm=pickle.load(open('znorm2.pkl', 'rb'))
    xte_psq=norm.transform(xtest['payment_sequential'].values.reshape(-1,1))
    
    norm=pickle.load(open('znorm6.pkl', 'rb'))
    xte_phr=norm.transform(xtest['f13_order_purchase_hour'].values.reshape(-1,1))
    

    
    
    #here we are vectorising the text data using tfidf such that we are ignoring the terms which are appearing less than 3 times
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer =pickle.load(open('zvectorizer.pkl', 'rb')) 
    xte_rcm=vectorizer.transform(xtest["review_comment_message"].values)
    
    #we are stacking the horizontally the all the vectors formed
    from scipy.sparse import hstack
    xte=hstack((xte_pcm,xte_pty,xte_os,xte_pnl,xte_pdl,xte_apv,xte_psq,xte_pv,xte_phr,xte_pri,xte_pw,xte_pi,xte_dad,xte_wpr,xte_opd,xtedon,xte_po,xte_rcm)).tocsr()
    
    if count==9:
      val=model.predict(xte)
    if val==1:
        return "positive review given"
    if val==0:
        return "negative review given"




In [8]:
pred=final_1('pos_1.xlsx')
print("*"*10)
print("\noutput is",pred)

data retained after doing doing data cleaning 100.0


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
100%|██████████| 1/1 [00:00<00:00, 1101.16it/s]


**********

output is positive review given


In [9]:
def final_2(uploaded_file):
    count=0
    xls = pd.ExcelFile(uploaded_file)
    geo= pd.read_excel(xls,'geo',index_col="Unnamed: 0")
    item=pd.read_excel(xls,'item',index_col="Unnamed: 0")
    customer= pd.read_excel(xls,'customer',index_col="Unnamed: 0")
    review=pd.read_excel(xls,'review',index_col="Unnamed: 0")
    orders= pd.read_excel(xls,'orders',index_col="Unnamed: 0")
    products=pd.read_excel(xls,'products',index_col="Unnamed: 0")
    sellers= pd.read_excel(xls,'sellers',index_col="Unnamed: 0")
    payment=pd.read_excel(xls,'payment',index_col="Unnamed: 0")
    name_translation=pd.read_excel(xls,'name_translation',index_col="Unnamed: 0")
    
    df=pd.merge(orders,payment,on="order_id",how="outer")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,customer,on="customer_id",how="outer")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,item,on="order_id",how="outer")
    df.drop_duplicates(inplace=True)
    df.drop_duplicates(subset=["order_id"],inplace=True)
    df=pd.merge(df,products,on="product_id",how="outer")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,review,on="order_id",how="outer")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,sellers,on="seller_id",how="inner")
    df.drop_duplicates(inplace=True)
    df=pd.merge(df,name_translation,on="product_category_name",how="outer")
    df.drop_duplicates(inplace=True)

    df1=df.copy()
    df1["order_approved_at"].fillna(value=df1["order_purchase_timestamp"],inplace=True)
    df1["order_delivered_customer_date"].fillna(value=df1["order_estimated_delivery_date"],inplace=True)
    df1["price"].fillna(value=df1["price"].median(),inplace=True)
    df1["freight_value"].fillna(value=df1["freight_value"].median(),inplace=True)
    df1["product_name_lenght"].fillna(value=df1["product_name_lenght"].median(),inplace=True)
    df1["product_description_lenght"].fillna(value=df1["product_description_lenght"].median(),inplace=True)
    df1["product_photos_qty"].fillna(value=df1["product_photos_qty"].median(),inplace=True)
    df1["product_weight_g"].fillna(value=df1["product_weight_g"].median(),inplace=True)
    df1["product_height_cm"].fillna(value=df1["product_height_cm"].median(),inplace=True)
    df1["product_length_cm"].fillna(value=df1["product_length_cm"].median(),inplace=True)
    df1["product_width_cm"].fillna(value=df1["product_width_cm"].median(),inplace=True)
    df1["payment_value"].fillna(value=df1["payment_value"].median(),inplace=True)
    df1["review_score"].fillna(value=df1["review_score"].median(),inplace=True)
    df1["review_comment_message"].fillna("nao_reveja",inplace=True)
    df1["payment_sequential"].fillna(value=df1["payment_sequential"].value_counts().index[0],inplace=True)
    df1["payment_type"].fillna(value=df1["payment_type"].value_counts().index[0],inplace=True)

    df1["payment_installments"].fillna(value=df1["payment_installments"].value_counts().index[0],inplace=True)
    df1["product_category_name"].fillna(value=df1["product_category_name"].value_counts().index[21],inplace=True)
    df1["product_category_name_english"].fillna(value=df1["product_category_name_english"].value_counts().index[21],inplace= True)
    df1["order_item_id"].fillna(value=df1["order_item_id"].value_counts().index[0],inplace=True)
    df1["seller_zip_code_prefix"].fillna(value=df1["seller_zip_code_prefix"].value_counts().index[0],inplace=True)
    df1["seller_city"].fillna(value=df1["seller_city"].value_counts().index[0],inplace=True)
    df1["seller_state"].fillna(value=df1["seller_state"].value_counts().index[0],inplace=True)
    df1.drop(columns=["review_answer_timestamp","review_creation_date","review_id","review_comment_title","order_delivered_carrier_date","product_category_name"],axis=1,inplace=True)
    df1.dropna(inplace=True)
    df1.drop_duplicates(subset=["order_id","customer_id","order_status","order_status","order_approved_at","order_delivered_customer_date","product_weight_g","product_category_name_english","review_comment_message"],inplace=True)
    print("data retained after doing doing data cleaning {:.1f}".format((df1.shape[0]/df.shape[0])*100))
    df1["review_score"]=df1["review_score"].apply(lambda x: 0 if x < 3 else 1)
    df1["order_purchase_timestamp"]=pd.to_datetime(df1["order_purchase_timestamp"])
    df1["order_approved_timestamp"]=pd.to_datetime(df1["order_approved_at"])
    df1["order_purchase_date"]=df1.order_purchase_timestamp.dt.date
    df1["approval_date"]=df1["order_approved_timestamp"].dt.date
    df1["order_estimated_delivery_date"]=pd.to_datetime(df1.order_estimated_delivery_date).dt.date
    df1["order_delivered_customer_date"]=pd.to_datetime(df1.order_delivered_customer_date).dt.date 



    import nltk
    nltk.download("stopwords")
    from nltk.corpus import stopwords
    from tqdm import tqdm
    from nltk.stem import RSLPStemmer
    import re
    nltk.download("rslp")
    stopwords_portugues=stopwords.words("portuguese")
    stopwords_portugues.remove("nem")
    stopwords_portugues.remove("não")
    stemmer=RSLPStemmer()
    preprocessed_reviews = []
    for sentance in tqdm(df1['review_comment_message'].values):
        sentance = re.sub(r"http\S+", "", sentance)
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        sentance=re.sub("[nN][ãaAÃ][oO][Ññ]|[nN]","Negação",sentance)
        sentance=sentance.replace("\\r",'')
        sentance=sentance.replace("\\n",'')
        sentance = re.sub('[^A-Za-z]+',' ',sentance)
        sentance = ' '.join(a for a in sentance.split() if a.lower not in stopwords_portugues)
        sentance = ' '.join(stemmer.stem(e.lower())for e in sentance.split())
        
        preprocessed_reviews.append(sentance.strip())


    df1["review_comment_message"]=preprocessed_reviews
    f10=df1.groupby("order_id").count()["product_id"]
    f10.df=pd.DataFrame()
    f10.df["order_id"]=f10.index
    f10.df["F10_number_of_product_ordered"]=f10.values
    f9=df1.groupby("product_id").count()["seller_id"]
    f9.df=pd.DataFrame()
    f9.df["product_id"]=f9.index
    f9.df["number_of_sellers"]=f9.values
    df1["f16_purchase_month"]=(df1["order_purchase_timestamp"]).apply(lambda x: x.month)
    df1["f14_delivery_days"]=(df1["order_delivered_customer_date"]-df1["order_purchase_date"]).dt.days
    df1=pd.merge(df1,f9.df,on="product_id")
    df1=pd.merge(df1,f10.df,on="order_id")
    
    df1["f1_delivery_estimated_days"]=(df1["approval_date"]-df1["order_estimated_delivery_date"]).dt.days
    df1["f2_delivery_actual_days"]=(df1["approval_date"]-df1["order_delivered_customer_date"]).dt.days
    df1["f3_on_time_or_late"]=(df1["f1_delivery_estimated_days"] > df1["f2_delivery_actual_days"]).astype("int")
    df1["f4_discount_or_not"]=(df1["payment_value"]-(df1["price"]+df1["freight_value"])).apply(lambda x: 1 if x < 0 else 0)#converted to boolean 
    df1["f5_purchase_approved_delay"]=(df1["approval_date"]-df1["order_purchase_date"]).dt.days
    df1["f6_Average_product_value_per_order"]=df1.price/(df1.F10_number_of_product_ordered)
    df1["f7_order_frieght_value"]=df1.freight_value/df1.price
    df1["f8_purchase_day_week"]=df1.order_purchase_timestamp.dt.dayofweek
    df1["f11_order_purch_day"]=df1.order_purchase_timestamp.dt.day
    df1["f12_words_per_review"]=df1.review_comment_message.apply(lambda x: len(x.split(" ")))
    df1["f13_order_purchase_hour"]=df1["order_purchase_timestamp"].apply(lambda x: x.hour)
    df1["f15_week_delivery"]=df1["order_purchase_timestamp"].apply(lambda x: x.dayofweek)
    df2=df1.copy()
    df2=df2[["order_id","product_description_lenght","product_name_lenght","order_status","payment_type","product_category_name_english","payment_installments","product_weight_g","price","payment_sequential","product_photos_qty","f13_order_purchase_hour","f6_Average_product_value_per_order","payment_value","f11_order_purch_day","f12_words_per_review",'f2_delivery_actual_days','f3_on_time_or_late', 'f4_discount_or_not','F10_number_of_product_ordered','review_score','review_comment_message']]
    df2 = df2.loc[:,~df2.columns.duplicated()].copy()
    y=df2["review_score"].values
    x=df2.drop("review_score",axis=1)

    from sklearn.model_selection import train_test_split
    xtrain,xtest,ytrain,ytest=cf1,cf2,cf3,list(cf4["class"].values)
    xtedon=xtest[['f3_on_time_or_late', 'f4_discount_or_not']]
    
    
    norm=pickle.load(open('znorm1.pkl', 'rb'))
    xte_apv=norm.transform(xtest['f6_Average_product_value_per_order'].values.reshape(-1,1))
    count+=1
    
    
    norm=pickle.load(open('znorm3.pkl', 'rb'))
    xte_pdl=norm.transform(xtest["product_description_lenght"].values.reshape(-1,1))
    count+=1
        
    norm=pickle.load(open('znorm4.pkl', 'rb'))
    xte_pnl=norm.transform(xtest["product_name_lenght"].values.reshape(-1,1))
    count+=1
    
    norm=pickle.load(open('znorm5.pkl', 'rb'))
    xte_pv=norm.transform(xtest["payment_value"].values.reshape(-1,1))
    count+=1
    
    norm=pickle.load(open('znorm7.pkl', 'rb'))
    xte_pri=norm.transform(xtest['price'].values.reshape(-1,1))
    count+=1
  
    norm=pickle.load(open('znorm8.pkl', 'rb'))
    xte_pw=norm.transform(xtest['product_weight_g'].values.reshape(-1,1))
    count+=1
    
    norm=pickle.load(open('znorm9.pkl', 'rb'))
    xte_pi=norm.transform(xtest['payment_installments'].values.reshape(-1,1))
    count+=1
   
    #here we are normalising the numerical features as so that all come to same scale
    norm=pickle.load(open('znorm10.pkl', 'rb'))
    xte_dad=norm.transform(xtest["f2_delivery_actual_days"].values.reshape(-1,1))
     

    norm=pickle.load(open('norm11.pkl', 'rb'))
    xte_opd=norm.transform(xtest["f11_order_purch_day"].values.reshape(-1,1))
    
    norm=pickle.load(open('znorm12.pkl', 'rb'))
    xte_wpr=norm.transform(xtest["f12_words_per_review"].values.reshape(-1,1))
    count+=1
    norm=pickle.load(open('znorm13.pkl', 'rb'))
    xte_po=norm.transform(xtest['F10_number_of_product_ordered'].values.reshape(-1,1))
    count+=1
  
    vect=pickle.load(open('zvect.pkl', 'rb'))
    xte_os=vect.transform(xtest["order_status"].values)



    vect=pickle.load(open('zvect1.pkl', 'rb'))
    xte_pty=vect.transform(xtest["payment_type"].values)


    vect=pickle.load(open('zvect2.pkl', 'rb'))
    xte_pcm=vect.transform(xtest["product_category_name_english"].values)

    norm=pickle.load(open('znorm2.pkl', 'rb'))
    xte_psq=norm.transform(xtest['payment_sequential'].values.reshape(-1,1))
    
    norm=pickle.load(open('znorm6.pkl', 'rb'))
    xte_phr=norm.transform(xtest['f13_order_purchase_hour'].values.reshape(-1,1))
    

    model=pickle.load(open('zstack_model.pkl', 'rb'))
    
    #here we are vectorising the text data using tfidf such that we are ignoring the terms which are appearing less than 3 times
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizer =pickle.load(open('zvectorizer.pkl', 'rb')) 
    xte_rcm=vectorizer.transform(xtest["review_comment_message"].values)
    
    #we are stacking the horizontally the all the vectors formed
    from scipy.sparse import hstack
    xte=hstack((xte_pcm,xte_pty,xte_os,xte_pnl,xte_pdl,xte_apv,xte_psq,xte_pv,xte_phr,xte_pri,xte_pw,xte_pi,xte_dad,xte_wpr,xte_opd,xtedon,xte_po,xte_rcm)).tocsr()
    y_true=ytest
    print(len(ytest),xte.shape)
    if count==9:
      y_pred=model.predict(xte)
      
    from sklearn.metrics import f1_score
    score=f1_score(y_true, y_pred, average='micro')
    return score







In [ ]:
prediction=final_2("combine.xlsx")
print()
print("\nf1 score is",prediction)

data retained after doing doing data cleaning 99.7


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
100%|██████████| 98896/98896 [00:44<00:00, 2199.89it/s]


30638 (30638, 2151)


f1 score is 0.9201971408055356
